In [ ]:
!gdown https://drive.google.com/uc?id=1P12YAYLOg8T7x7beOkHozaPFsBRTFz5w

Downloading...
From: https://drive.google.com/uc?id=1P12YAYLOg8T7x7beOkHozaPFsBRTFz5w
To: /content/GTZAN Dataset - Music Genre Classification.zip
100% 1.30G/1.30G [00:20<00:00, 62.9MB/s]


In [ ]:
!unzip GTZAN\ Dataset\ -\ Music\ Genre\ Classification.zip

In [95]:
import glob
import json
import os

import IPython.display as ipd
import librosa
import numpy as np

In [96]:
SAMPLE_RATE = 22050
DURATION = 30  # 30SEC
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [97]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    data = {
        "mapping": [],
        "mfcc": [],
        "labels": []
    }

    for i, (dirpath, dirname, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath is not dataset_path:
            dirpath_components = dirpath.split('/')
            semantic_label = dirpath_components[-1]
            # if semantic_label == 'jazz':
            #     continue
            data["mapping"].append(semantic_label)
            print(f"\n Processing {semantic_label}")

            num_samples_per_segent = int(SAMPLES_PER_TRACK / num_segments)
            expected_num_mfcc_vectos_per_segment = np.ceil(
                (num_samples_per_segent / hop_length))

            for f in filenames:
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

                for s in range(num_segments):
                    start_sample = num_samples_per_segent * s
                    finish_sample = start_sample + num_samples_per_segent

                    mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                                sr=sr,
                                                n_fft=n_fft,
                                                n_mfcc=n_mfcc,
                                                hop_length=hop_length

                                                )
                    mfcc = mfcc.T

                    if len(mfcc) == expected_num_mfcc_vectos_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i - 1)

#                         print(f"{file_path}, segment:{s+1}")
    return data

In [100]:
jazz = glob.glob('/content/Data/genres_original/jazz/*.wav')
ipd.Audio(jazz[0])

In [ ]:
DATASET_PATH = "/content/Data/genres_original"
JSON_PATH = "data.json"
data = save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)


 Processing blues

 Processing pop

 Processing jazz

 Processing .ipynb_checkpoints

 Processing classical

 Processing metal

 Processing rock

 Processing hiphop

 Processing disco

 Processing country


In [ ]:
JSON_PATH = "data.json"
with open(JSON_PATH, "w") as fp:
    json.dump(data, fp, indent=4)

In [ ]:
def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return X, y

In [ ]:
x, y = load_data("/content/data.json")

Data succesfully loaded!
